In [23]:
import numpy as np
import mmh3
from scipy import integrate

https://en.wikipedia.org/wiki/HyperLogLog

In [336]:
class HyperLogLog:
    
    def __init__(self, m: int):
        self._M = [0] * m
    
    @staticmethod
    def _get_alpha(m):
    
        def f(u):
            return np.log2((2 + u) / (1 + u)) ** m

        return (m * integrate.quad(f, a=0, b=np.inf)[0]) ** -1

    @staticmethod
    def _hash(value: str) -> str:
        # mmh3 returns 32-bit hash, format it with leading zeros
        return format(mmh3.hash(value, signed=False), "032b")
    
    def add(self, value: str):
        m = len(self._M)
        b = np.log2(m).astype("int")
        
        # binary hash as a string
        hashed = self._hash(value)
        
        # which register to update -> first b bits
        j = int(hashed[:b], base=2)
        
        # find the position of the leftmost 1 in the rest of the bits
        try:
            ro = next(i for i, v in enumerate(hashed[b:]) if v == "1") + 1
        except StopIteration:
            ro = len(hashed[b:]) + 1
        
        # update the j-th register if ro is greater than the current state
        self._M[j] = max(ro, self._M[j])
        
    def count(self) -> int:
        m = len(self._M)
        alpha = self._get_alpha(m)
        Z = sum(2 ** -Mj for Mj in self._M) ** -1
        return round(alpha * Z * m ** 2)

In [333]:
data = [str(uuid.uuid4()) for _ in range(1000000)]

In [334]:
hll = HyperLogLog(m=32)
for value in data:
     hll.add(value)

In [335]:
hll.count()

959653